In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%pip install kagglehub
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install tensorflow
%pip install matplotlib
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import kagglehub
import random
import numpy as np
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from multiprocessing import Pool, cpu_count

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
path = kagglehub.dataset_download("tboyle10/medicaltranscriptions")
print("Path to dataset files:", path)

dataset = pd.read_csv(path + "/mtsamples.csv")
print("Head: ", dataset.head)

Path to dataset files: /kaggle/input/medicaltranscriptions
Head:  <bound method NDFrame.head of       Unnamed: 0                                        description  \
0              0   A 23-year-old white female presents with comp...   
1              1           Consult for laparoscopic gastric bypass.   
2              2           Consult for laparoscopic gastric bypass.   
3              3                             2-D M-Mode. Doppler.     
4              4                                 2-D Echocardiogram   
...          ...                                                ...   
4994        4994   Patient having severe sinusitis about two to ...   
4995        4995   This is a 14-month-old baby boy Caucasian who...   
4996        4996   A female for a complete physical and follow u...   
4997        4997   Mother states he has been wheezing and coughing.   
4998        4998   Acute allergic reaction, etiology uncertain, ...   

                medical_specialty                  

In [4]:
dataset.dropna(subset=['transcription', 'medical_specialty'], inplace=True)

dataset = dataset[['transcription', 'medical_specialty']]

specialty_counts = dataset['medical_specialty'].value_counts()
# valid_specialties = specialty_counts[specialty_counts >= 50].index
# dataset = dataset[dataset['medical_specialty'].isin(valid_specialties)]

dataset['medical_specialty'] = dataset['medical_specialty'].str.strip()

excluded_specialties = [
    'Surgery',
    'SOAP / Chart / Progress Notes',
    'Office Notes',
    'Consult - History and Phy.',
    'Emergency Room Reports',
    'Discharge Summary',
    'Pain Management',
    'General Medicine',
    'Radiology',
]

dataset = dataset[~dataset['medical_specialty'].isin(excluded_specialties)]

category_mapping = {
    'Neurosurgery': 'Neurology',
    'Nephrology': 'Urology',
}

dataset['medical_specialty'] = dataset['medical_specialty'].replace(category_mapping)

for i, (category_name, category) in enumerate(dataset.groupby("medical_specialty")):
    print(f"Category {i}: {category_name}: {len(category)}")

Category 0: Allergy / Immunology: 7
Category 1: Autopsy: 8
Category 2: Bariatrics: 18
Category 3: Cardiovascular / Pulmonary: 371
Category 4: Chiropractic: 14
Category 5: Cosmetic / Plastic Surgery: 27
Category 6: Dentistry: 27
Category 7: Dermatology: 29
Category 8: Diets and Nutritions: 10
Category 9: ENT - Otolaryngology: 96
Category 10: Endocrinology: 19
Category 11: Gastroenterology: 224
Category 12: Hematology - Oncology: 90
Category 13: Hospice - Palliative Care: 6
Category 14: IME-QME-Work Comp etc.: 16
Category 15: Lab Medicine - Pathology: 8
Category 16: Letters: 23
Category 17: Neurology: 317
Category 18: Obstetrics / Gynecology: 155
Category 19: Ophthalmology: 83
Category 20: Orthopedic: 355
Category 21: Pediatrics - Neonatal: 70
Category 22: Physical Medicine - Rehab: 21
Category 23: Podiatry: 47
Category 24: Psychiatry / Psychology: 53
Category 25: Rheumatology: 10
Category 26: Sleep Medicine: 20
Category 27: Speech - Language: 9
Category 28: Urology: 237


In [5]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym.lower() != word.lower():
                synonyms.add(synonym)
    return list(synonyms)

def synonym_replacement(text, n=None):
    words = word_tokenize(text)
    new_words = words.copy()
    eligible_words = list(set([word for word in words if word.isalpha()]))
    random.shuffle(eligible_words)

    if n is None:
        n = random.randint(1, min(3, len(eligible_words)))

    num_replaced = 0
    for word in eligible_words:
        synonyms = get_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = [synonym if w == word else w for w in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return ' '.join(new_words)

def word_dropout(text, dropout_prob=0.1):
    words = word_tokenize(text)
    new_words = [word for word in words if random.random() > dropout_prob]
    return ' '.join(new_words) if new_words else text

def random_swap(text, n=1):
    words = word_tokenize(text)
    if len(words) < 2:
        return text
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

def augment_text_randomly(text):
    aug_functions = [
        synonym_replacement,
        word_dropout,
        random_swap
    ]
    num_augs = random.randint(1, 2)
    selected_augs = random.sample(aug_functions, num_augs)
    for aug in selected_augs:
        text = aug(text)
    return text

def augment_class(label_samples_tuple):
    label, samples_needed, class_samples = label_samples_tuple
    samples = class_samples.sample(n=samples_needed, replace=True)
    texts = samples['transcription'].tolist()

    with Pool(cpu_count()) as p:
        augmented_texts = p.map(augment_text_randomly, texts)

    return pd.DataFrame({
        'transcription': augmented_texts,
        'medical_specialty': label
    })

specialty_counts = dataset['medical_specialty'].value_counts()
max_count = specialty_counts.max()

tasks = []
for label, count in specialty_counts.items():
    if count < max_count:
        samples_needed = max_count - count
        class_samples = dataset[dataset['medical_specialty'] == label]
        tasks.append((label, samples_needed, class_samples))

augmented_dfs = [augment_class(task) for task in tasks]

augmented_df = pd.concat(augmented_dfs, ignore_index=True)
dataset = pd.concat([dataset, augmented_df], ignore_index=True)


In [6]:
from sklearn.model_selection import train_test_split

lemmatizer = None
stop_words = None

def init_worker():
    global lemmatizer, stop_words
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.strip().lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

def apply_multiprocessing(series, func, workers=None):
    with Pool(processes=workers or cpu_count(), initializer=init_worker) as pool:
        results = pool.map(func, series)
    return results

# Use multiprocessing to speed it up
dataset['processed_transcription'] = apply_multiprocessing(dataset['transcription'], clean_text)

X_train, X_test, y_train, y_test = train_test_split(
    dataset['processed_transcription'], dataset['medical_specialty'], test_size=0.2, random_state=42, stratify=dataset['medical_specialty']
)

In [7]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(texts):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
        with torch.no_grad():
            outputs = bert(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(cls_embedding)
    return np.array(embeddings)

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y_train)

X_train_bert = get_bert_embeddings(X_train)
X_test_bert = get_bert_embeddings(X_test)

y_train_cat = to_categorical(y_encoded)
y_test_encoded_cat = label_encoder.transform(y_test)
y_test_cat = to_categorical(y_test_encoded_cat)

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(768,)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_bert, y_train_cat, epochs=20, batch_size=32, validation_split=0.2)

y_pred_probs = model.predict(X_test_bert)
y_pred = np.argmax(y_pred_probs, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test_encoded_cat, y_pred, digits=4))

2025-04-24 19:26:51.614880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745522811.811727      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745522811.870743      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch 1/20


I0000 00:00:1745525473.998688     343 service.cc:148] XLA service 0x7a23a00066c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745525473.999340     343 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745525473.999361     343 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745525474.221910     343 cuda_dnn.cc:529] Loaded cuDNN version 90300


117/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0977 - loss: 3.2584  

I0000 00:00:1745525475.832746     343 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.1414 - loss: 3.0983 - val_accuracy: 0.5163 - val_loss: 1.9486
Epoch 2/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4274 - loss: 1.9911 - val_accuracy: 0.6446 - val_loss: 1.3322
Epoch 3/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5600 - loss: 1.5109 - val_accuracy: 0.7276 - val_loss: 1.0396
Epoch 4/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6384 - loss: 1.2350 - val_accuracy: 0.7387 - val_loss: 0.9387
Epoch 5/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6927 - loss: 1.0442 - val_accuracy: 0.7758 - val_loss: 0.8072
Epoch 6/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7323 - loss: 0.8942 - val_accuracy: 0.7695 - val_loss: 0.7998
Epoch 7/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7408 - loss: 0.8586 - val_accuracy: 0.7811 - val_loss: 0.7400
Epoch 8/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7502 - loss: 0.8231 - val_accuracy: 0.7933 - val

In [8]:
from transformers import BertTokenizer
import pickle

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
with open('bert_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [9]:
from sklearn.preprocessing import LabelEncoder
import pickle

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


In [10]:
model.save('bert_ffnn_model.h5')  # You can later load it with keras.models.load_model()


In [11]:
import numpy as np

np.save('X_train_bert.npy', X_train_bert)
np.save('X_test_bert.npy', X_test_bert)
